In [16]:
import time as t
import datetime as dt
import pandas as pd
import os
import sys
sys.path.insert(1, '../')
from utils.common import *
from utils.blockchain import *

In [17]:
TAKE_PROFIT = 1
STOP_LOSS = 1
PRIORITY_FEE = 5000

In [25]:
dt.datetime.fromtimestamp(1714518779, tz=dt.timezone.utc)

datetime.datetime(2024, 4, 30, 23, 12, 59, tzinfo=datetime.timezone.utc)

In [26]:
dt.datetime.fromtimestamp(1714518779, tz=dt.timezone.utc)

datetime.datetime(2024, 4, 30, 23, 12, 59, tzinfo=datetime.timezone.utc)

In [29]:
with SQLiteDB('../dbs/calls.db') as conn:
    query = "SELECT *  FROM calls"
    db_tokens = pd.read_sql_query(query, conn)
db_tokens

,id,date,address,expected_launch_time,expected_launch_time_ts,s_mm2,s_ma2,s_fa2,s_q,s_sni,mcap_num,liq_num,launched,launch_time,buy,buy_time,buy_price
0,231561,2024-05-01 00:11:19,GXFB8MkB3d7ryfdVACDegDYR27R7nkNVc8riNLbf2aRL,11 seconds ago,2024-05-01 00:11:08,1,1,0,0,3,12660,25320,1.0,2024-05-01 00:11:00+00:00,None,None,None
1,231562,2024-05-01 00:13:06,6U13s9V6J8cpU9EMvi1XgN2qxQg4NBn5hDzLeynCNZK9,7 seconds ago,2024-05-01 00:12:59,0,0,0,0,1,1440,2790,1.0,2024-05-01 00:13:00+00:00,None,None,None
2,231567,2024-05-01 00:19:57,7Mj8faMw52px3GkVYZj8pkez5BYu5ZyUKfdKs3gBzbuV,10 seconds ago,2024-05-01 00:19:47,0,0,0,0,2,564,1020,1.0,2024-05-01 00:19:00+00:00,None,None,None
3,231568,2024-05-01 00:20:27,DYZFhaR4PxtcwRtsQqXy4nBXdwK9uCHbkkxv2uHujocy,8 seconds ago,2024-05-01 00:20:19,0,0,0,2,0,12790,25330,1.0,2024-05-01 00:20:00+00:00,None,None,None
4,231569,2024-05-01 00:20:46,BA5G7y1D5K1bCxzWG6YaWe32fd6yLxWd89k9RT2dW5ys,In 8 minutes,2024-05-01 00:28:46,0,0,0,0,2,502,506,NaN,None,None,None,None
5,231573,2024-05-01 00:27:19,F4orxbDvyAjRHgt2q36frNvwbhb4nVQA2c9RbcgNNgjc,17 seconds ago,2024-05-01 00:27:02,1,0,0,1,2,504,1010,NaN,None,None,None,None


In [18]:
tow = getSPLtokens(wallet)

In [19]:
tow

,pubkey,account,token_address,decimals,amount,amount_int
0,9bTDu83hE37NSCvkNrXfCyNLHzQc7rV8XF7KyqqzaUQ8,"{'lamports': 2039280, 'data': {'program': 'spl...",EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,6,19820014,19.820014
1,5qRpRknfKus1qiNL1iQU92atWtktTsnBMoHppLfMzinP,"{'lamports': 2039280, 'data': {'program': 'spl...",AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy,9,5938092146082,5938.092146
2,7izYUZtuGMoXCC8Wx2uHpX8Ui4png7X6S64WnLs36nFQ,"{'lamports': 2039280, 'data': {'program': 'spl...",8AntP7Hxft8Yjr52UQ3RnBHU4njTkDFDho97gE4vRzkt,9,902936636325,902.936636


In [20]:
tracked_tokens = pd.merge(
    tow,
    db_tokens[['address','buy_price']],
    how='inner', left_on='token_address', right_on='address'
)
cur_prices = check_multi_price(tracked_tokens['token_address'].to_list())
cur_prices = pd.DataFrame().from_dict(cur_prices,orient='index')
cur_prices = cur_prices.reset_index().rename(columns={'index':'address'})
tracked_tokens = pd.merge(tracked_tokens, cur_prices, how='inner', on='address')
tracked_tokens['liq_req'] = tracked_tokens['amount_int']*tracked_tokens['price']
tracked_tokens['price_change'] = tracked_tokens['price']/tracked_tokens['buy_price']
tracked_tokens['sell_tag'] = np.where((tracked_tokens['liq_req'] < tracked_tokens['liquidity']) &
                                      ((tracked_tokens['price_change']>=TAKE_PROFIT) |
                                       (tracked_tokens['price_change']<-STOP_LOSS)), True, False)

In [25]:
print(tracked_tokens[['address','price_change']])

                                        address  price_change
0  AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy      0.994101


In [11]:
for i, row in tracked_tokens[tracked_tokens['sell_tag']==True].iterrows():
    print(dt.datetime.now(), 'Attempt to SELL: ', i, row['address'], '. X:', row['price_change'])
    result, txid = tx_procedure(wallet=wallet, asset_in=row['address'], asset_out=SOL_ca,
                                mode='sell', fee=PRIORITY_FEE)
    print('Result: ', result)
    if result == {'Ok': None}:
        print('SUCCESS BUY')
        ## UPDATE TO DB
        with SQLiteDB('dbs/calls.db') as conn:
            update_statement = f"""
            UPDATE calls
            SET buy = {False},
            WHERE address = "{row['address']}"
            """
            conn.execute(update_statement)
        print('DB updated with sell data!')

2024-04-28 19:35:57.484823 Attempt to SELL:  0 AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy
attempt to sell 101191.563343478 of AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy
Success Quote!
fee will be:  5000
Success Txgen!
Success sign!


RuntimeError: asyncio.run() cannot be called from a running event loop